# Data lake access

### With a SAS

In [33]:
import pandas as pd

In [ ]:
# Generated SAS for csv
sas_token = '<sas_token>'
sas_uri ='<sas_url>'

df = pd.read_csv(sas_uri) 
df

### With SDK - gen 1
https://medium.com/azure-data-lake/using-jupyter-notebooks-and-pandas-with-azure-data-lake-store-48737fbad305

In [5]:
# !pip install azure-mgmt-resource
# !pip install azure-mgmt-datalake-store
# !pip install azure-datalake-store

In [6]:
# !pip list | grep 'azure-datalake-store\|azure-mgmt-datalake-store\|azure-mgmt-resource'

In [38]:
from azure.datalake.store import core, lib, multithread
# https://github.com/Azure/azure-data-lake-store-python

In [ ]:
from azure.common.credentials import ServicePrincipalCredentials
# In entra create a new app and for that app a secret

TENANT_ID = '<tenant_id>' 
CLIENT_SECRET = '<clinet_secret>' # client secret value
CLIENT_ID = '<client_id>' # app id

token = lib.auth(tenant_id=TENANT_ID,
                 client_secret=CLIENT_SECRET,
                 client_id=CLIENT_ID)
token.token

In [ ]:
adlsFileSystemClient = core.AzureDLFileSystem(token, store_name='mystrgaccricgen1')
adlsFileSystemClient

# # Read a file into pandas dataframe
with adlsFileSystemClient.open('/contg1/frutas.csv', 'rb') as f:
    df = pd.read_csv(f) 

# # Show the dataframe
df

### With SDK - gen2
https://peter-hoffmann.com/2020/azure-data-lake-storage-gen-2-with-python.html

In [ ]:
# !pip install azure-storage-file-datalake --pre
# https://pypi.org/project/azure-storage-file-datalake/

In [ ]:
# Get connection string from: stg account Access Keys 
connection_string = "<connection_string>"

In [ ]:
# Show files/folders in container
from azure.storage.filedatalake import FileSystemClient

file_system = FileSystemClient.from_connection_string(
    connection_string, 
    file_system_name="cont2")

paths = file_system.get_paths()

for path in paths:
    print(path.name + '\n')

In [ ]:
# Read a file
from azure.storage.filedatalake import DataLakeFileClient
from io import StringIO

file = DataLakeFileClient.from_connection_string(connection_string,
                                                 file_system_name="cont2", file_path="frutas.csv")


data = file.download_file().read().decode('utf-8')
print(data)
df = pd.read_csv(StringIO(data))
df

In [ ]:
s_buf = StringIO()
df.to_csv(s_buf)

file = DataLakeFileClient.from_connection_string(connection_string,
                                                 file_system_name="cont2", file_path="frutas2.csv")
file.create_file()
file.append_data(data, offset=0)
file.flush_data(len(data))